# 0. Installing Transformers and Importing Dependencies

In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.5 MB 30.6 MB/s 
     |████████████████████████████████| 7.6 MB 71.1 MB/s 
     |████████████████████████████████| 163 kB 64.6 MB/s 


# 1. Load Summarization Pipeline

In [ ]:
!pip install sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.3 MB 21.4 MB/s 


In [ ]:
# from transformers import PegasusTokenizer, PegasusForConditionalGeneration

# model = PegasusForConditionalGeneration.from_pretrained("google/pegasus-xsum")
# tokenizer = PegasusTokenizer.from_pretrained("google/pegasus-xsum")

In [ ]:
from transformers import BartTokenizer, BartForConditionalGeneration
model = BartForConditionalGeneration.from_pretrained("facebook/bart-large-cnn")
tokenizer = BartTokenizer.from_pretrained("facebook/bart-large-cnn")


Downloading:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/899k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

In [ ]:
from transformers import pipeline
summarizer = pipeline("summarization",model=model, tokenizer=tokenizer)

# 4. Summarize Text

In [ ]:
import os

In [ ]:
lectures_list = []
for root, dirs, files in os.walk("/content/data"):
    for file in files:
        if file.endswith('.txt'):
            with open(os.path.join(root, file), 'r') as f:
                article = f.read()
                lectures_list.append(article)

In [ ]:
#lectures_list

In [ ]:
from tqdm import tqdm
max_chunk = 420

lectures_summaries = {}

for lecture in tqdm(lectures_list):
  original_lecture = lecture
  lecture = lecture.replace('.', '.<eos>')
  lecture = lecture.replace('?', '?<eos>')
  lecture = lecture.replace('!', '!<eos>')

  sentences = lecture.split('<eos>')
  current_chunk = 0 
  chunks = []
  for sentence in sentences:
      if len(chunks) == current_chunk + 1: 
          if len(chunks[current_chunk]) + len(sentence.split(' ')) <= max_chunk:
              chunks[current_chunk].extend(sentence.split(' '))
          else:
              current_chunk += 1
              chunks.append(sentence.split(' '))
      else:
          print(current_chunk)
          chunks.append(sentence.split(' '))

  for chunk_id in range(len(chunks)):
      chunks[chunk_id] = ' '.join(chunks[chunk_id])

  res = summarizer(chunks, max_length=120, min_length=30, do_sample=False)
  summary = ' '.join([summ['summary_text'] for summ in res])

  lectures_summaries[original_lecture] = summary

  0%|          | 0/13 [00:00<?, ?it/s]

0


  8%|▊         | 1/13 [02:29<29:58, 149.91s/it]

0


 15%|█▌        | 2/13 [05:04<27:59, 152.66s/it]

...

Your max_length is set to 120, but you input_length is only 113. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=56)
 85%|████████▍ | 11/13 [24:30<04:41, 140.90s/it]

0


 92%|█████████▏| 12/13 [26:12<02:09, 129.04s/it]

0


100%|██████████| 13/13 [30:21<00:00, 140.09s/it]


In [ ]:
import pandas as pd
df = pd.DataFrame(lectures_summaries.items(), columns=['Lecture', 'Summary'])

In [ ]:
df.head()

,Lecture,Summary
0,Welcome back to Medical Engineering and today ...,In the next two videos we want to talk about a...
1,Welcome back to Medical Engineering. Today we ...,Welcome back to Medical Engineering. Today we ...
2,Welcome back to medical engineering so today w...,The term endoscope emerges from Greek “endon” ...
3,Welcome back to Medical Engineering. Today we ...,Welcome back to Medical Engineering. Today we ...
4,Welcome back to Medical Engineering. So you’ve...,In this video we look at Magnetic Resonance Im...


In [ ]:
df.to_csv("BART_Summaries.csv")